In [49]:
import pitct
import os

# --- 阶段 0: 项目设置 ---
# 创建一个工作目录来存放 pitct 生成的文件
if not os.path.exists('llm_controller_project'):
    os.makedirs('llm_controller_project')
pitct.init('llm_controller_project', overwrite=True)

print("--- 项目：基于DES的LLM交互行为形式化调控 ---")
print("--- 场景：技术支持机器人 ---")

# --- 阶段 1: 场景定义与 Plant 建模 (G) ---

# 状态定义 (States)
# 0: S0_Idle (空闲)
# 1: S1_ProblemReceived (已接收问题)
# 2: S2_InfoGathering (收集中)
# 3: S3_SolutionProposed (已提出方案)
# 4: S4_Escalated (已升级)
# 5: S5_Resolved (已解决)
states_g = {
    0: "S0_Idle", 1: "S1_ProblemReceived", 2: "S2_InfoGathering",
    3: "S3_SolutionProposed", 4: "S4_Escalated", 5: "S5_Resolved"
}
statenum_g = len(states_g)

# 事件定义 (Events)
# 可控事件 (Controllable, 'c'): Prompts
# 不可控事件 (Uncontrollable, 'u'): LLM/User Responses
# 格式: (source_state, event_name, dest_state, controllability_flag)
trans_g = [
    # 流程开始
    (0, 'p_ask_problem', 1, 'c'),
    # 从接收问题到收集信息
    (1, 'p_ask_details', 2, 'c'),
    # 在问题接收阶段，用户/LLM 也可能感到困惑
    (1, 'r_is_confused', 1, 'u'),   # 用户/LLM 感到困惑，停留在当前阶段
    # 收集信息阶段，LLM/用户的可能回应
    (2, 'r_provides_info', 3, 'u'), # 用户提供信息，进入方案提出阶段
    (2, 'r_is_confused', 2, 'u'),   # 用户/LLM 感到困惑，停留在当前阶段
    # 提出方案后
    (3, 'r_confirms_solved', 5, 'u'), # 用户确认解决
    (3, 'r_requests_escalation', 4, 'u'), # 用户要求升级
    (3, 'r_is_confused', 3, 'u'),   # 用户在方案提出阶段也可能困惑
    # 允许直接从问题接收后就升级（这是我们之后要禁止的行为）
    (1, 'p_escalate', 4, 'c'),
    # 允许在提出方案后，由机器人主动升级
    (3, 'p_escalate', 4, 'c'),
    # 升级后，用户也可能困惑
    (4, 'r_is_confused', 4, 'u'),   # 用户在升级阶段也可能困惑
    # 解决或升级后，可以回到空闲状态
    (4, 'p_end_session', 0, 'c'),
    (5, 'r_is_confused', 5, 'u'),   # 用户在解决阶段也可能困惑
    (5, 'p_end_session', 0, 'c'),
]

# 标记状态 (Marker States): 代表任务成功或稳定结束的状态
marker_g = [0, 5] 

# 使用 pitct 创建 Plant 自动机 G
pitct.create('G', statenum_g, trans_g, marker_g)
print("\n[1] Plant 'G' 创建成功。代表所有可能的交互流程。")
# 可视化 Plant
model_G = pitct.display_automaton('G', color=True)
model_G.save('G_plant.png')
print("    -> Plant 模型图已保存为 'G_plant.png'")



--- 项目：基于DES的LLM交互行为形式化调控 ---
--- 场景：技术支持机器人 ---

[1] Plant 'G' 创建成功。代表所有可能的交互流程。
    -> Plant 模型图已保存为 'G_plant.png'


In [50]:

# --- 阶段 2: 规范设计与控制器合成 (E & C) ---

# 规范: "在事件 r_provides_info 发生之前，事件 p_escalate 绝不能发生"

# 状态:
# 0: E0_BeforeSolution (方案未提出)
# 1: E1_AfterSolution (方案已提出)
# 2: E2_Trap (违规陷阱)
states_e = {0: "E0_BeforeSolution", 1: "E1_AfterSolution", 2: "E2_Trap"}
statenum_e = len(states_e)

event_controllability = {name: flag for _, name, _, flag in trans_g}
all_events = event_controllability.keys()

trans_e = []
for event in all_events:
    flag = event_controllability[event]
    
    # --- 状态 E0 (方案未提出) 的逻辑 ---
    if event == 'p_escalate':
        # 违规行为: 在方案提出前升级 -> 进入陷阱
        trans_e.append((0, event, 2, flag))
    elif event == 'r_provides_info':
        # 关键进展: 方案已提出 -> 进入 E1
        trans_e.append((0, event, 1, flag))
    else:
        # 其他事件: 保持在 E0
        trans_e.append((0, event, 0, flag))
        
    # --- 状态 E1 (方案已提出) 的逻辑 ---
    # 在此之后，所有行为都是允许的，状态保持不变
    trans_e.append((1, event, 1, flag))
    
    # --- 状态 E2 (陷阱) 的逻辑 ---
    # 一旦进入陷阱，永远停留在陷阱
    trans_e.append((2, event, 2, flag))

# 只有非陷阱状态是标记状态
marker_e = [0, 1]


# 创建 Specification 自动机 E
pitct.create('E', statenum_e, trans_e, marker_e)
print("\n[2] Specification 'E' 创建成功。定义了期望的交互规则。")
model_E = pitct.display_automaton('E', color=True)
model_E.save('E_spec.png')
print("    -> 规范模型图已保存为 'E_spec.png'")

# 合成 Supervisor
print("\n[3] 正在合成 Supervisor 'C'...")
# 首先计算同步积 S = G || E
pitct.sync('S', 'G', 'E')
pitct.display_automaton('S', color=True).save('S_sync.png')
# 然后计算最优控制器 C = supcon(G, S)
pitct.supcon('C', 'G', 'S')
print("    -> Supervisor 'C' 合成完毕！")
model_C = pitct.display_automaton('C', color=True)
model_C.save('C_supervisor.png')
print("    -> Supervisor 模型图已保存为 'C_supervisor.png'")




[2] Specification 'E' 创建成功。定义了期望的交互规则。
    -> 规范模型图已保存为 'E_spec.png'

[3] 正在合成 Supervisor 'C'...
    -> Supervisor 'C' 合成完毕！
    -> Supervisor 模型图已保存为 'C_supervisor.png'


In [51]:
# --- 阶段 3: 原型实现与闭环测试 (完全修正版) ---

print("\n" + "="*50)
print("--- 阶段 3: 启动与模拟LLM的闭环控制交互 ---")
print("="*50)

# 获取控制器 C 的所有转移，以便在循环中重复使用
all_c_transitions = pitct.trans('C')

# 构建 C 状态到 G 状态的映射
# 策略：通过追踪 C 中的转移事件，推断对应的 G 状态
# 因为 C 是 G 和 E 的同步积的子集，我们需要通过事件序列来映射
c_state_to_g_state = {}

# 使用 BFS 从初始状态开始，追踪每个 C 状态对应的 G 状态
from collections import deque
queue = deque([(0, 0)])  # (c_state, g_state)
visited = {0}
c_state_to_g_state[0] = 0

while queue:
    c_state, g_state = queue.popleft()
    
    # 获取从当前 C 状态出发的所有转移
    for c_trans in all_c_transitions:
        if c_trans[0] == c_state:
            event = c_trans[1]
            next_c_state = c_trans[2]
            
            # 在 G 中查找相同事件的转移
            next_g_state = None
            for g_trans in trans_g:
                if g_trans[0] == g_state and g_trans[1] == event:
                    next_g_state = g_trans[2]
                    break
            
            if next_g_state is not None and next_c_state not in visited:
                c_state_to_g_state[next_c_state] = next_g_state
                visited.add(next_c_state)
                queue.append((next_c_state, next_g_state))

print("[DEBUG] 从控制器 C 到 Plant G 的状态映射:")
print(c_state_to_g_state)

prompt_texts = {
    'p_ask_problem': "Hello! I'm a support bot. Please describe your problem.",
    'p_ask_details': "Thank you. Can you provide more details?",
    'p_escalate': "I will escalate this issue to a human agent.",
    'p_end_session': "Session is ending. Goodbye."
}

def mock_llm(prompt, current_state_name):
    print(f"[LLM Input] Prompt: '{prompt}' (from state: {current_state_name})")
    if "describe your problem" in prompt: return "My computer won't turn on.", "r_is_confused"
    if "more details" in prompt: return "I've already checked the power cable.", "r_provides_info"
    if "ending" in prompt or "Goodbye" in prompt: return "Thank you!", "r_confirms_solved"
    return "I am confused.", "r_is_confused"

# ============================ 核心修正逻辑开始 ============================

# 1. 初始状态总是 0
current_state_c = 0

while True:
    # 将 C 的当前状态映射回 G 的状态以显示给用户
    g_state_id = c_state_to_g_state.get(current_state_c)
    if g_state_id is None:
        print(f"错误：无法映射控制器状态 {current_state_c} 到 G 的状态。会话终止。")
        break
    current_state_name = states_g[g_state_id]
    
    print(f"\n==> 当前状态: {current_state_name} (G:{g_state_id}, C:{current_state_c}) <==")

    # 2. 从所有转移中筛选出当前状态的出口转移
    trans_from_current = [t for t in all_c_transitions if t[0] == current_state_c]

    if not trans_from_current:
        print("会话结束（到达终端状态）。")
        break

    allowed_prompts = [t[1] for t in trans_from_current if t[1] in prompt_texts]
    
    if not allowed_prompts:
        print("当前状态下没有可执行的动作，等待外部事件...")
        uncontrollable_trans = [t for t in trans_from_current if t[1] not in prompt_texts]
        if not uncontrollable_trans:
            print("会话结束（无可用转移）。")
            break
        event_to_fire = uncontrollable_trans[0][1]
        next_state_c = uncontrollable_trans[0][2]
        print(f"自动触发不可控事件: {event_to_fire}")
        current_state_c = next_state_c
        continue

    print("--- 请选择一个操作 (Prompt) ---")
    for i, p_label in enumerate(allowed_prompts):
        print(f"  {i+1}: {prompt_texts[p_label]} ({p_label})")

    try:
        choice = int(input("输入选项编号: ")) - 1
        chosen_prompt_label = allowed_prompts[choice]
    except (ValueError, IndexError):
        print("无效输入，请重试。")
        continue

    # 3. 查找并执行转移
    next_state_c = -1
    for t in trans_from_current:
        if t[1] == chosen_prompt_label:
            next_state_c = t[2]
            break
    
    print(f"--- [Action] 执行 '{chosen_prompt_label}'。")
    current_state_c = next_state_c
    
    # 如果执行的是 p_end_session，直接结束会话，不再调用 LLM
    if chosen_prompt_label == 'p_end_session':
        print("--- [Session] 会话正常结束。")
        break
    
    # 调用LLM
    g_state_id_for_llm = c_state_to_g_state.get(current_state_c, -1)
    llm_response_text, llm_response_event = mock_llm(
        prompt_texts[chosen_prompt_label], states_g.get(g_state_id_for_llm, "Unknown")
    )
    print(f"    [LLM Output] Response: '{llm_response_text}' (分类为: '{llm_response_event}')")

    # 根据LLM响应更新状态
    next_state_after_llm = -1
    # 需要重新筛选从新状态出发的转移
    trans_from_new_state = [t for t in all_c_transitions if t[0] == current_state_c]
    for t in trans_from_new_state:
        if t[1] == llm_response_event:
            next_state_after_llm = t[2]
            break

    if next_state_after_llm != -1:
        print(f"--- [Event] 发生 '{llm_response_event}'。")
        current_state_c = next_state_after_llm
    else:
        print(f"警告：在控制器状态 {current_state_c} 发生意外事件 '{llm_response_event}'！会话终止。")
        break

# ============================ 核心修正逻辑结束 ============================


--- 阶段 3: 启动与模拟LLM的闭环控制交互 ---
[DEBUG] 从控制器 C 到 Plant G 的状态映射:
{0: 0, 1: 1, 2: 2, 3: 3, 4: 5, 5: 4, 6: 0, 7: 1, 8: 2}

==> 当前状态: S0_Idle (G:0, C:0) <==
--- 请选择一个操作 (Prompt) ---
  1: Hello! I'm a support bot. Please describe your problem. (p_ask_problem)
--- [Action] 执行 'p_ask_problem'。
[LLM Input] Prompt: 'Hello! I'm a support bot. Please describe your problem.' (from state: S1_ProblemReceived)
    [LLM Output] Response: 'My computer won't turn on.' (分类为: 'r_is_confused')
--- [Event] 发生 'r_is_confused'。

==> 当前状态: S1_ProblemReceived (G:1, C:1) <==
--- 请选择一个操作 (Prompt) ---
  1: Thank you. Can you provide more details? (p_ask_details)
--- [Action] 执行 'p_ask_details'。
[LLM Input] Prompt: 'Thank you. Can you provide more details?' (from state: S2_InfoGathering)
    [LLM Output] Response: 'I've already checked the power cable.' (分类为: 'r_provides_info')
--- [Event] 发生 'r_provides_info'。

==> 当前状态: S3_SolutionProposed (G:3, C:3) <==
--- 请选择一个操作 (Prompt) ---
  1: I will escalate this issue to 